In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.paths.directory import *


/home/cbisot/pycode/MscThesis/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
x = {i : set() for i in range(660000)}
getsize(x)

182011604

In [5]:
x = {i : 2 for i in range(660000)}
getsize(x)

39451604

In [2]:
import sys
from types import ModuleType, FunctionType
from gc import get_referents

# Custom objects know their class.
# Function objects seem to know way too much, including modules.
# Exclude modules as well.
BLACKLIST = type, ModuleType, FunctionType


def getsize(obj):
    """sum size of object & members."""
    if isinstance(obj, BLACKLIST):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            if not isinstance(obj, BLACKLIST) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
        objects = get_referents(*need_referents)
    return size

In [2]:
directory = directory_project
# update_plate_info(directory)

In [2]:
directory = '/projects/0/einf914/agg/'
update_plate_info(directory)

analysed:   0%|          | 0/528 [00:00<?, ?it/s]

In [3]:
all_folders = get_current_folders(directory)

In [4]:
folders = all_folders.loc[all_folders['Plate']=="808"]
print(len(folders))

91


In [5]:
folders = all_folders.loc[all_folders['Temp'].isin(['30to25'])]
print(len(folders))
folders = folders.loc[folders['/Analysis/skeleton_pruned_compressed.mat']==True]
print(len(folders))
folders = folders.loc[folders['/Analysis/skeleton_realigned_compressed.mat']==True]
print(len(folders))
# folders = folders.loc[folders['Plate'].isin(['99','92','90','138',"112",'127'])]
# print(len(folders))
folders = folders.loc[folders['Plate'].isin(['48'])]
print(len(folders))
# folders = folders.loc[folders['/Analysis/nx_graph_pruned_labeled.p']==True]
# print(len(folders))
print(len(folders["Plate"].unique()))
# folders = folders.loc[folders['Plate'].isin(["112",'127'])]


962
921
918
113
1


In [6]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,Temperature,InocDate
5898,/projects/0/einf914/data/20211003_1937_Plate35,808,35,'Carrot','A5','001P100N','Y','-',20210928,True,...,False,True,True,True,True,"03.10.2021, 19:37:",20211003_1937_Plate35,NaN,NaN,NaN
5936,/projects/0/einf914/data/20211001_0003_Plate35,808,35,'Carrot','A5','001P100N','Y','-',20210928,True,...,False,True,True,True,True,"01.10.2021, 00:03:",20211001_0003_Plate35,NaN,NaN,NaN
5957,/projects/0/einf914/data/20211003_1737_Plate35,808,35,'Carrot','A5','001P100N','Y','-',20210928,True,...,False,True,True,True,True,"03.10.2021, 17:37:",20211003_1737_Plate35,NaN,NaN,NaN
6010,/projects/0/einf914/data/20211004_1135_Plate35,808,35,'Carrot','A5','001P100N','Y','-',20210928,True,...,False,True,True,True,True,"04.10.2021, 11:35:",20211004_1135_Plate35,NaN,NaN,NaN
6011,/projects/0/einf914/data/20211004_2137_Plate35,808,35,'Carrot','A5','001P100N','Y','-',20210928,True,...,False,True,True,True,True,"04.10.2021, 21:37:",20211004_2137_Plate35,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7619,/projects/0/einf914/data/20211005_1736_Plate35,808,35,'Carrot','A5','001P100N','Y','-',20210928,True,...,False,True,True,True,True,"05.10.2021, 17:36:",20211005_1736_Plate35,NaN,NaN,NaN
7623,/projects/0/einf914/data/20211005_0337_Plate35,808,35,'Carrot','A5','001P100N','Y','-',20210928,True,...,False,True,True,True,True,"05.10.2021, 03:37:",20211005_0337_Plate35,NaN,NaN,NaN
7645,/projects/0/einf914/data/20210930_0352_Plate35,808,35,'Carrot','A5','001P100N','Y','-',20210928,True,...,False,True,True,True,True,"30.09.2021, 03:52:",20210930_0352_Plate35,NaN,NaN,NaN
7671,/projects/0/einf914/data/20211011_0816_Plate35,808,35,'Carrot','A5','001P100N','Y','-',20210928,True,...,False,True,True,True,True,"11.10.2021, 08:16:",20211011_0816_Plate35,NaN,NaN,NaN


***Create graphs***

In [20]:
num_parallel = 128
time = '5:00:00'
args=[directory]
run_parallel('extract_nx_graph.py',args, folders, num_parallel, time,'extract_nx',cpus = 128,node ='fat')

Submitted batch job 507919
Submitted batch job 507921


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [69]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned.p'])&(all_folders['Plate'].isin(plates))]
len(folders)

336

***Extract Width***

In [6]:
num_parallel = 128
time = '24:00:00'
skip = False
resolution = 50
args = [directory,skip,resolution]
run_parallel('extract_width.py',args,folders,num_parallel,time,'extract_width2',cpus=32,node="fat")

Submitted batch job 508441
Submitted batch job 508442


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 508443
Submitted batch job 508444
Submitted batch job 508445


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [41]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(1-all_folders['/Analysis/nx_graph_pruned_width.p'])&(all_folders['Plate'].isin(plates))&(all_folders['/Analysis/nx_graph_pruned.p'])]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

0

In [4]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned_width.p'])&(all_folders['Plate'].isin(plates))]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

336

***Identify Nodes***

In [7]:
num_parallel = 32
time = '24:00:00'
limit = 55
skip = False
args=[directory,limit,skip]
run_parallel_all_time('extract_nodes.py',args,folders,num_parallel,time,'node_id',cpus = 32,node="fat")

Submitted batch job 535639


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [60]:
plates = [22,10,76,26,152,102]

folders = all_folders.loc[(all_folders['/Analysis/nx_graph_pruned_labeled.p'])&(all_folders['Plate'].isin(plates))]
# folders.loc[folders['folder'] == '20210110_1523_Plate30']
# int(list(folders['folder'])[0].split('_')[-1][5:])
len(folders)

239

***Hyphae extraction***

In [6]:
num_parallel = 32
time = '8:00:00'
limit = 55
version = 11
labeled = True
args = [directory,limit,version,labeled]
run_parallel_all_time('hyphae_extraction.py',args,folders,num_parallel,time,'hyphae',cpus = 32,node="fat")

Submitted batch job 551041


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 960000 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
